In [6]:

import argparse
import os
import random
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.utils as utils
import torch.nn.functional as F
from models import G_CIFAR10,G_MNIST,D_CIFAR10,D_MNIST
from tensorboard_logger import configure, log_value

from data import generate_c_data, CIFAR10_Complementary

from train_test import train,train_data_g,test,test_acc



from torchvision import datasets, transforms

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enbaled = True




def args():
    FLAG = argparse.ArgumentParser(description='ACGAN Implement With Pytorch.')
    FLAG.add_argument('--dataset', default='MNIST', help='CIFAR10 | MNIST')
    FLAG.add_argument('--savingroot', default='../result', help='path to saving.')
    FLAG.add_argument('--dataroot', default='data', help='path to dataset.')
    FLAG.add_argument('--manual_seed', default=42, help='manual seed.')
    FLAG.add_argument('--p1', default= 0.0, type=float, help='p1')
    FLAG.add_argument('--p2', default=1.0, type=float, help='p2')
    FLAG.add_argument('--image_size', default=32, help='image size.')
    FLAG.add_argument('--batch_size', default=128, help='batch size.')
    FLAG.add_argument('--num_workers', default=2, help='num workers.')
    FLAG.add_argument('--num_epoches', default=20, help='num workers.')
    FLAG.add_argument('--nc', default=1, help='channel of input image.')
    FLAG.add_argument('--nz', default=64, help='length of noize.')
    FLAG.add_argument('--ndf', default=64, help='number of filters.')
    FLAG.add_argument('--ngf', default=64, help='number of filters.')
    arguments = FLAG.parse_args()
    return arguments


##############################################################################





assert torch.cuda.is_available(), '[!] CUDA required!'





def embed_z(opt):
    fixed = Variable(torch.Tensor(100, opt.nz).normal_(0, 1)).cuda()
    return fixed



def train_gan(opt):

    os.makedirs(os.path.join(opt.savingroot,opt.dataset,str(opt.p1 * 100) + '%complementary/' + str(opt.p1) + '_images'), exist_ok=True)
    os.makedirs(os.path.join(opt.savingroot,opt.dataset,str(opt.p1 * 100) + '%complementary/' + str(opt.p1) +  '_chkpts'), exist_ok=True)
    if not os.path.exists(os.path.join(opt.savingroot,opt.data_r,'data','processed/training'+str(opt.p1)+str(opt.p2)+'.pt')):
        generate_c_data(opt)
    else:
        print('data exits')

    #Build networ
    if opt.data_r == 'MNIST':
        netd = D_MNIST(opt.ndf, opt.nc, num_classes=10).cuda()
        netg = G_MNIST( opt.nz, opt.ngf, opt.nc).cuda()
    elif opt.data_r == 'CIFAR10':
        netd = D_CIFAR10(opt.ndf, opt.nc, num_classes=10).cuda()
        netg = G_CIFAR10(opt.nz, opt.ngf, opt.nc).cuda()



    optd = optim.Adam(netd.parameters(), lr=0.0002,
                      betas=(0.5, 0.999))  # optim.SGD(netd.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)#
    optg = optim.Adam(netg.parameters(), lr=0.0002,
                      betas=(0.5, 0.999))  # optim.SGD(netg.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)#

    print('training_start')
    step = 0
    acc = []

    # configure(os.path.join(opt.savingroot,opt.dataset,str(opt.p1 * 100) + '%complementary/' +  str(opt.p1) + '/logs'), flush_secs=5)
    test_loader = torch.utils.data.DataLoader(
        CIFAR10_Complementary(os.path.join(opt.savingroot,opt.data_r,'data'), train=False, transform=transforms.Compose([
            transforms.Resize(opt.image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])), batch_size=128, num_workers=2)

    fixed = embed_z(opt)

    for epoch in range(opt.num_epoches):
        print(f'Epoch {epoch:03d}.')
        dataset = CIFAR10_Complementary(os.path.join(opt.savingroot,opt.data_r,'data'), transform=tsfm, p1=opt.p1,p2=opt.p2)
        # dataset = dset.CIFAR10(root=opt.dataroot, download=True, transform=tsfm)
        loader = torch.utils.data.DataLoader(dataset, batch_size=opt.batch_size, shuffle=True, num_workers=2
                                             )
        # if epoch % 40 == 0 and epoch != 0:
        #     for param_group in optd.param_groups:
        #         param_group['lr'] = param_group['lr'] / 10
        #         print(param_group['lr'])
        step = train(netd,netg,optd,optg,loader,epoch,step,opt)

        test(netg, fixed, epoch, opt)
        acc.append(test_acc(netd, test_loader))

    f = open(os.path.join(opt.savingroot,opt.dataset,str(opt.p1 * 100) + '%complementary/'  + 'acc.txt'), 'a')
    for cont in acc:
        f.writelines(str(cont) + '\n')

    f.close()

def train_f_data(opt):
    os.makedirs(os.path.join(opt.savingroot, opt.dataset, str(opt.p1 * 100) + '%complementary/' + str(opt.p1) + '_chkpts_fake_data'),exist_ok=True)


    if opt.data_r == 'MNIST':
        netd = D_MNIST(opt.ndf, opt.nc, num_classes=10).cuda()
        netg = G_MNIST( opt.nz, opt.ngf, opt.nc).cuda()
    elif opt.data_r == 'CIFAR10':
        netd = D_CIFAR10(opt.ndf, opt.nc, num_classes=10).cuda()
        netg = G_CIFAR10(opt.nz, opt.ngf, opt.nc).cuda()
    print(os.path.join(opt.savingroot, opt.dataset, str(opt.p1 * 100) + '%complementary/' + str(
            opt.p1) + f'_chkpts/g_{(opt.num_epoches-1):03d}.pth'))
    netg.load_state_dict(torch.load(os.path.join(opt.savingroot, opt.dataset, str(opt.p1 * 100) + '%complementary/' + str(
            opt.p1) + f'_chkpts/g_{(opt.num_epoches-1):03d}.pth')))
    netg.eval()



    # netd = MobileNetV2().cuda()
    # netd = VGG('VGG19').cuda()
    # netd = ResNet18().cuda()




    # netg.load_state_dict(model.state_dict())

    optd = optim.Adam(netd.parameters(), lr=0.0002,
                      betas=(0.5, 0.999))  # optim.SGD(netd.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)#

    print('training_start')
    step = 0
    acc = []


    test_loader = torch.utils.data.DataLoader(
        CIFAR10_Complementary(os.path.join(opt.savingroot,opt.data_r,'data'), train=False, transform=transforms.Compose([
            transforms.Resize(opt.image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])), batch_size=128, num_workers=2)

    fixed = embed_z(opt)

    for epoch in range(opt.num_epoches*2):
        print(f'Epoch {epoch:03d}.')
        # if epoch % 40 == 0 and epoch != 0:
        #     for param_group in optd.param_groups:
        #         param_group['lr'] = param_group['lr'] / 10
        #         print(param_group['lr'])
        step = train_data_g(netd,netg,optd,epoch,step,opt)

        test(netg, fixed, epoch, opt)
        acc.append(test_acc(netd, test_loader))

    f = open(os.path.join(opt.savingroot,opt.dataset,str(opt.p1 * 100) + '%complementary/' + 'acc_f_train.txt'), 'a')
    for cont in acc:
        f.writelines(str(cont) + '\n')

    f.close()



if __name__ == '__main__':

    opt = args()
    opt.data_r = opt.dataset



    if opt.data_r == 'MNIST':
        tsfm = transforms.Compose([
            transforms.Resize(opt.image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])
    else:
        tsfm = transforms.Compose([
            transforms.Resize(opt.image_size),
            transforms.RandomCrop(opt.image_size, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])






    torch.cuda.manual_seed(opt.manual_seed)
    opt.dataset = os.path.join(opt.dataset, opt.dataset + '_' + str(opt.p2))

    configure(os.path.join(opt.savingroot, opt.dataset, str(opt.p1 * 100) + '%complementary/' + str(opt.p1) + '/logs'),
              flush_secs=5)

    train_gan(opt)
    train_f_data(opt)



usage: ipykernel_launcher.py [-h] [--dataset DATASET]
                             [--savingroot SAVINGROOT] [--dataroot DATAROOT]
                             [--manual_seed MANUAL_SEED] [--p1 P1] [--p2 P2]
                             [--image_size IMAGE_SIZE]
                             [--batch_size BATCH_SIZE]
                             [--num_workers NUM_WORKERS]
                             [--num_epoches NUM_EPOCHES] [--nc NC] [--nz NZ]
                             [--ndf NDF] [--ngf NGF]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1435621388/jupyter/kernel-98d6dda4-9fcb-4150-9348-32405e60e1e2.json


SystemExit: 2

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

import torch.nn as nn
import torch.nn.functional as F


class G_MNIST(nn.Module):

    def __init__(self, nz, ngf, nc):
        super(G_MNIST, self).__init__()

        self.embed = nn.Embedding(10, nz)

        self.conv1 = nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0)
        self.bn1 = nn.BatchNorm2d(ngf * 8)

        self.conv2 = nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(ngf * 4)

        self.conv3 = nn.ConvTranspose2d(ngf * 4, ngf * 1, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ngf * 1)


        self.conv5 = nn.ConvTranspose2d(ngf * 1, nc, 4, 2, 1)

        self.relu = nn.ReLU(True)
        self.tanh = nn.Tanh()

        self.__initialize_weights()

    def forward(self, z,label):
        input = z.mul_(self.embed(label))
        x = input.view(input.size(0), -1, 1, 1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)


        x = self.conv5(x)
        output = self.tanh(x)
        return output

    def __initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, 0.02)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.normal_(1.0, 0.02)
                m.bias.data.fill_(0)


class D_MNIST(nn.Module):

    def __init__(self, ndf, nc, num_classes=10):
        super(D_MNIST, self).__init__()
        self.ndf = ndf
        self.lrelu = nn.ReLU()
        self.conv1 = nn.Conv2d(nc, ndf, 4, 2, 1)

        self.conv3 = nn.Conv2d(ndf , ndf * 4, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(ndf * 8)
        self.conv5 = nn.Conv2d(ndf * 8, ndf * 1, 4, 1, 0)
        self.gan_linear = nn.Linear(ndf * 1, 1)
        self.aux_linear = nn.Linear(ndf * 1, num_classes)

        self.sigmoid = nn.Sigmoid()
        self.__initialize_weights()

    def forward(self, input):

        x = self.conv1(input)
        x = self.lrelu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.lrelu(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.lrelu(x)

        x = self.conv5(x)
        x = x.view(-1, self.ndf * 1)
        c = self.aux_linear(x)

        s = self.gan_linear(x)
        s = self.sigmoid(s)
        return s.squeeze(1), c.squeeze(1)

    def __initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, 0.02)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.normal_(1.0, 0.02)
                m.bias.data.fill_(0)

class G_CIFAR10(nn.Module):

    def __init__(self, nz, ngf, nc):
        super(G_CIFAR10, self).__init__()
        self.embed = nn.Embedding(10, nz)
        self.conv1 = nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0)
        self.bn1 = nn.BatchNorm2d(ngf * 8)

        self.conv2 = nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(ngf * 4)

        self.conv3 = nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ngf * 2)

        self.conv4 = nn.ConvTranspose2d(ngf * 2, ngf * 1, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(ngf * 1)

        self.conv5 = nn.ConvTranspose2d(ngf, ngf * 1, 3, 1, 1)
        self.bn5 = nn.BatchNorm2d(ngf * 1)

        self.conv6 = nn.Conv2d(ngf * 1, nc, 3, 1, 1)

        self.relu = nn.ReLU(True)
        self.tanh = nn.Tanh()

        self.__initialize_weights()

    def forward(self, z,label):
        input = z.mul_(self.embed(label))
        x = input.view(input.size(0), -1, 1, 1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu(x)

        x = self.conv6(x)
        output = self.tanh(x)
        return output

    def __initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, 0.02)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.normal_(1.0, 0.02)
                m.bias.data.fill_(0)


class D_CIFAR10(nn.Module):

    def __init__(self, ndf, nc, num_classes=10):
        super(D_CIFAR10, self).__init__()
        self.ndf = ndf
        self.lrelu = nn.ReLU()
        self.conv0 = nn.Conv2d(nc, ndf, 3, 1, 1)
        self.bn0 = nn.BatchNorm2d(ndf)
        self.conv1 = nn.Conv2d(ndf, ndf, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(ndf)
        self.conv2 = nn.Conv2d(ndf, ndf * 2, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(ndf * 2)
        self.conv3 = nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(ndf * 8)
        self.conv5 = nn.Conv2d(ndf * 8, ndf * 1, 4, 1, 0)
        self.gan_linear = nn.Linear(ndf * 1, 1)
        self.aux_linear = nn.Linear(ndf * 1, num_classes)

        self.sigmoid = nn.Sigmoid()
        self.__initialize_weights()

    def forward(self, input):

        x = self.conv0(input)
        x = self.bn0(x)
        x = self.lrelu(x)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.lrelu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.lrelu(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.lrelu(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.lrelu(x)

        x = self.conv5(x)
        x = x.view(-1, self.ndf * 1)
        c = self.aux_linear(x)

        s = self.gan_linear(x)
        s = self.sigmoid(s)
        return s.squeeze(1), c.squeeze(1)

    def __initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, 0.02)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.normal_(1.0, 0.02)
                m.bias.data.fill_(0)


NameError: name 'main' is not defined